In [1]:
import alpaca_trade_api as tradeapi
import numpy as np
from scipy import optimize as opt
import pandas as pd
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from bs4 import BeautifulSoup
import requests
from math import log
from sklearn.linear_model import LinearRegression
import datetime 
import workdays as wkdy

/mnt/c/Users/natha/Documents/TourdeFinance/trading_env/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
with open("keys.txt") as keys_file :
    GENERAL_API_KEY = keys_file.readline().strip()
    SECRET_API_KEY = keys_file.readline().strip()
ALPACA_URL_BASE  = "https://paper-api.alpaca.markets"
ALPACA_DATA_URL_BASE = "https://data.alpaca.markets/v1"

In [3]:
api  = tradeapi.REST(GENERAL_API_KEY, SECRET_API_KEY, ALPACA_URL_BASE, api_version = 'v2')
account = api.get_account()

In [4]:
#use get_available_datasets() to see all the different Fama French datasets available
ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')
ff5 = ff5[0][::-1] #change it from a dict to a dataframe and then change it 
                   #so it's from newest to oldest
ff5.reset_index(inplace=True)

In [5]:
ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2020-02-28,-0.78,0.06,-0.78,0.32,-0.41,0.006
1,2020-02-27,-4.22,0.69,0.14,-0.35,-0.09,0.006
2,2020-02-26,-0.52,-0.79,-1.26,-0.52,-0.12,0.006
3,2020-02-25,-3.09,-0.34,-0.72,-0.63,0.03,0.006
4,2020-02-24,-3.38,0.15,-0.04,-0.37,0.20,0.006
...,...,...,...,...,...,...,...
1225,2015-04-17,-1.23,-0.43,0.23,0.13,0.10,0.000
1226,2015-04-16,-0.08,-0.10,-0.20,-0.23,-0.21,0.000
1227,2015-04-15,0.57,0.27,0.32,-0.19,-0.07,0.000
1228,2015-04-14,0.11,-0.13,0.19,0.09,0.09,0.000


In [6]:
# get sp500 tickers, code courtesy of Will Everett
page = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
soup = BeautifulSoup(page.content, 'html.parser')
sp500_tickers = [row.find('td').get_text().replace('\n','')
                 for row in soup.find('tbody').find_all('tr')[1:]]
sp500_tickers.remove('BRK.B')
sp500_tickers.remove('BF.B')
sp500_tickers.sort()

In [7]:
days_of_data = 1000
open_close = pd.DataFrame()
for i in range(0,len(sp500_tickers),100) :
    data = api.get_barset(sp500_tickers[i:i+99],'day',limit=days_of_data)
    for ticker in sp500_tickers[i:i+99] :
        opin = []
        close = []
        for day in range(days_of_data - 1, -1, -1) :
            try :
                if (data[ticker][day].o == 0) :
                    opin.append(np.nan)
                else :
                    opin.append(data[ticker][day].o)
                close.append(data[ticker][day].c)
            except :
                opin.append(np.nan)
                close.append(np.nan)
        open_close[ticker + ' Open'] = opin
        open_close[ticker + ' Close'] = close

In [8]:
open_close

,A Open,A Close,AAL Open,AAL Close,AAP Open,AAP Close,AAPL Open,AAPL Close,ABBV Open,ABBV Close,...,XYL Open,XYL Close,YUM Open,YUM Close,ZBRA Open,ZBRA Close,ZION Open,ZION Close,ZTS Open,ZTS Close
0,76.43,78.280,12.64,12.5100,104.62,106.98,268.70,267.960,78.56,79.7250,...,69.68,69.34,77.59,80.25,198.83,201.26,30.00,31.0000,126.55,128.750
1,74.17,76.690,10.91,11.3300,101.83,102.48,262.74,266.040,76.44,78.5500,...,65.40,69.10,72.22,75.37,195.74,196.65,28.24,29.4300,126.36,127.170
2,76.61,74.030,11.07,10.2175,102.23,99.71,270.80,259.385,78.38,75.2700,...,67.31,66.18,73.99,71.47,199.70,192.36,28.10,28.0400,129.02,126.170
3,72.97,74.250,9.72,9.5000,89.44,95.80,250.90,262.285,75.74,75.6200,...,63.39,65.19,66.55,70.43,184.60,191.52,25.74,26.9200,119.58,126.740
4,71.71,70.400,10.34,9.3900,86.14,84.76,242.80,241.430,74.65,73.3508,...,63.12,60.53,65.71,63.30,173.02,177.57,25.21,24.6800,115.08,114.620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,41.35,41.595,36.79,36.7100,157.73,158.26,96.00,97.810,60.93,60.7100,...,42.84,42.91,81.65,81.24,65.18,63.66,26.42,27.4500,48.34,47.340
996,41.82,41.300,36.57,36.9200,157.59,157.40,103.91,104.300,56.78,60.9200,...,42.47,42.88,85.00,81.69,64.46,65.25,27.13,27.3201,48.54,48.645
997,41.71,41.630,37.46,36.3000,157.38,158.43,105.00,105.080,56.92,56.7800,...,42.29,42.21,82.50,82.57,64.59,64.70,26.74,27.2800,48.42,48.460
998,41.98,41.800,36.96,37.4900,158.40,157.16,105.02,105.680,57.07,57.2000,...,41.96,42.46,81.48,82.12,64.28,64.47,26.98,26.7200,48.56,48.480


In [9]:
for tkr in sp500_tickers :
    try :
        open_close[tkr + ' Open'][0]
    except :
        sp500_tickers.remove(tkr)

In [10]:
len(sp500_tickers)

498

In [11]:
#number of days between today and when the stock market got hit by COVID-19
today = datetime.date.today()
days_since_crash = wkdy.networkdays(datetime.date(2020, 2, 19), today)
days_since_crash

38

In [12]:
hit_hard = []
for tkr in sp500_tickers :
    dif = (open_close[tkr + ' Open'][days_since_crash] -
           open_close[tkr + ' Close'][0]) / open_close[tkr + ' Open'][days_since_crash]
    hit_hard.append(dif)

In [14]:
#finds the n greatest numbers in a list
def n_max(lis, n) :
    maxes = [] 
    for i in range(0, n):  
        max1 = 0
        for j in range(len(lis)):      
            if lis[j] >= max1: 
                try :
                    maxes.index(lis[j])
                except ValueError :
                    max1 = lis[j]
        maxes.append(max1)
    return maxes

In [15]:
discounted_ten_percent = n_max(hit_hard, 50)
discounted_indicies = []
for i in range(len(discounted_ten_percent)) :
    discounted_indicies.append(hit_hard.index(discounted_ten_percent[i]))
len(discounted_indicies)

50

In [16]:
portfolio_stocks = []
for index in discounted_indicies :
    portfolio_stocks.append(sp500_tickers[index])
portfolio_stocks

['IR',
 'NCLH',
 'CCL',
 'APA',
 'ARNC',
 'RCL',
 'OXY',
 'HAL',
 'MRO',
 'OKE',
 'ADS',
 'NBL',
 'UAL',
 'M',
 'DAL',
 'DVN',
 'AAL',
 'HP',
 'MPC',
 'BA',
 'KSS',
 'MGM',
 'FANG',
 'SLB',
 'ALK',
 'FTI',
 'SPG',
 'NOV',
 'GPS',
 'EXPE',
 'XRX',
 'DFS',
 'UNM',
 'SYF',
 'LYV',
 'TDG',
 'COTY',
 'DRI',
 'WYNN',
 'AIG',
 'PXD',
 'JWN',
 'TPR',
 'IVZ',
 'GE',
 'CMA',
 'MAR',
 'PVH',
 'LNC',
 'PHM']

In [17]:
#calculate the days between now and the fama french data
today = datetime.date.today()
ff5end_date = ff5['Date'][0]
#presidentsday = [datetime.date(2020, 2, 17)]
days_offset = wkdy.networkdays(ff5end_date.date(), today)

In [18]:
excess_returns = pd.DataFrame()
num_data = days_of_data - days_offset
excess_returns['Date'] = ff5['Date'][:num_data]
for tkr in portfolio_stocks :
    ex_returns = []
    for day in range(days_offset, days_of_data) :
        daily_ret = (open_close[tkr + ' Close'][day] -
                     open_close[tkr + ' Open'][day] -
                     ff5['RF'][day]) / open_close[tkr + ' Open'][day]
        ex_returns.append(daily_ret)
    excess_returns[tkr] = ex_returns

In [19]:
excess_returns

,Date,IR,NCLH,CCL,APA,ARNC,RCL,OXY,HAL,MRO,...,PXD,JWN,TPR,IVZ,GE,CMA,MAR,PVH,LNC,PHM
0,2020-02-28,-0.001841,-0.084580,-0.083660,-0.041804,0.000440,-0.089518,-0.057237,-0.043994,-0.067408,...,-0.041595,-0.017971,-0.043369,-0.030489,-0.036588,-0.026310,-0.048111,-0.044229,-0.029877,-0.014202
1,2020-02-27,-0.032424,-0.085355,-0.057000,-0.063363,-0.044759,-0.084537,-0.084876,-0.047419,-0.061684,...,-0.048881,-0.061840,-0.056434,-0.057957,-0.054804,-0.051545,-0.087477,-0.060398,-0.057293,-0.042756
2,2020-02-26,0.002427,-0.042841,-0.041036,-0.008383,0.020494,-0.033777,-0.025333,-0.020543,-0.018257,...,0.002019,-0.000026,-0.029852,-0.002143,0.014029,-0.017360,-0.014589,0.010249,-0.034962,-0.007807
3,2020-02-25,0.003404,-0.024231,-0.010116,0.008671,0.000120,-0.025230,-0.010247,-0.017837,-0.005517,...,0.008534,-0.001871,-0.016380,-0.022974,-0.016546,-0.007562,-0.014761,-0.014389,-0.007572,-0.006338
4,2020-02-24,0.005606,-0.058186,-0.002956,-0.000909,-0.012199,-0.009795,0.006066,-0.026021,-0.029953,...,-0.010319,0.014173,0.025053,0.012584,-0.010681,0.009777,-0.004718,0.010045,0.007497,0.003744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,2016-04-29,-0.000328,0.022777,0.007307,-0.022296,1.272341,0.019867,-0.001741,0.029340,0.005356,...,0.000943,0.016518,-0.006695,0.012141,0.000291,-0.001624,0.024295,-0.002838,-0.004795,-0.002704
965,2016-04-28,0.011756,-0.005718,-0.059813,0.016808,-0.014581,0.014223,0.012804,-0.000283,0.024769,...,0.037975,0.015376,-0.000273,-0.010911,0.002562,0.034524,-0.005950,-0.007655,0.013460,-0.009922
966,2016-04-27,-0.002488,0.011882,0.009824,-0.002566,-0.546303,0.013500,-0.007725,-0.017569,-0.029544,...,0.006551,0.010076,-0.013835,-0.001265,-0.005577,-0.007033,0.003431,0.002695,0.005520,0.000470
967,2016-04-26,-0.004865,-0.027209,-0.013446,0.003469,-0.024373,-0.035559,0.005748,0.016744,0.025698,...,0.009286,-0.014348,0.012228,-0.001886,-0.010968,0.025642,-0.024784,0.007047,0.028296,0.028916


In [53]:
#calculate the betas, this will be our A matrix
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
betas = pd.DataFrame()
X = pd.DataFrame(ff5[factors][0:num_data], columns=factors)
for tkr in portfolio_stocks :
    y = excess_returns[tkr]
    model = LinearRegression().fit(X, y)
    betas[tkr] = model.coef_

In [48]:
#calculate the premiums for each risk factor
#take a discounted average of the log of the returns, then exponentiate the average
wgts = []
discount_factor = 0.999
for i in range(num_data) :
    wgts.append(discount_factor ** i)
    
log_returns = pd.DataFrame()
for factor in factors :
    log_ret = []
    for i in range(num_data) :
        val = log(ff5[factor][i] + 100) #add 100 so they're all positive 
        log_ret.append(val)             
    log_returns[factor + ' Log Returns'] = log_ret

log_premiums = []
for factor in factors :
    lp = np.average(log_returns[factor + ' Log Returns'], weights = wgts)
    log_premiums.append(lp)

premiums = np.exp(log_premiums)
premiums -= 100

In [64]:
#calculate the expects returns for each stock
#these with be the coefficients of our objective function
expected_returns = []
for tkr in portfolio_stocks :
    exp_re = ff5['RF'][0] #expected returns is the risk-free rate, plus the betas times the premiums
    for factor in range(len(factors)) :
        exp_re += betas[tkr][factor]*premiums[factor]
    expected_returns.append(exp_re)

In [71]:
#multiply each of the expected returns and each of the betas by the current stock price
#also add the constraint that x * all the stock prices must be less than total capital
stock_prices = pd.DataFrame()
for tkr in portfolio_stocks :
    betas[tkr] = betas[tkr] * open_close[tkr + ' Close'][0]
    for i in range(len(portfolio_stocks)) :
        expected_returns[i] *= open_close[tkr + ' Close'][0]
        expected_returns[i] *= -1 #we have to reverse our obj funct because it minimizes
    stock_prices[tkr] = [open_close[tkr + ' Close'][0]]
betas = betas.append(stock_prices)

,IR,NCLH,CCL,APA,ARNC,RCL,OXY,HAL,MRO,OKE,...,PXD,JWN,TPR,IVZ,GE,CMA,MAR,PVH,LNC,PHM
0,-0.006673,0.009302,-0.012471,0.002745,-0.134196,-0.001003,-0.003643,0.005169,0.002089,-0.008401,...,-0.008643,-0.008404,-0.006181,0.007180,0.008897,0.027173,0.055883,0.042505,0.034885,-0.000028
1,-0.004154,-0.012533,0.015279,-0.001607,-0.255989,0.025549,0.002881,0.011697,0.009834,-0.006239,...,-0.063140,0.009815,0.004837,-0.006552,-0.008790,-0.008734,0.124413,-0.019177,-0.044434,-0.007490
2,0.000951,0.024909,0.031596,-0.004327,0.065820,0.032541,0.014510,0.005769,-0.000980,0.002731,...,0.042872,0.007542,-0.001575,0.015127,0.001307,0.010641,0.061488,-0.008062,0.034727,0.017137
3,-0.025949,0.017989,0.058768,0.004181,-0.048851,0.054764,-0.001379,0.011806,-0.003996,-0.001951,...,0.102927,0.013785,-0.000959,-0.022085,-0.012693,0.033367,0.275973,0.075067,0.017720,0.000041
4,-0.008760,-0.027509,-0.088601,0.014507,0.041257,0.010339,0.013737,-0.004691,-0.002889,0.025532,...,0.053680,-0.026126,-0.001717,-0.015013,-0.016347,0.003528,-0.040439,-0.019612,-0.046472,-0.039906
0,26.500000,13.114700,12.425000,8.190000,10.810000,40.220000,15.352400,8.210000,4.120000,28.380000,...,75.460000,21.330000,15.695000,9.870000,7.140000,34.750000,81.330000,49.520000,34.290000,26.340000


In [72]:
#last, create our b vector
#the amount of risk we're willing to take in each direction, along with our total capital
b = [.5,.5,-.2,0,0,1000]

In [80]:
#use the linear program solver
x = opt.linprog(c = expected_returns, A_ub = betas, b_ub = b)
x

     con: array([], dtype=float64)
     fun: -2.064619355802407e+71
 message: 'The algorithm terminated successfully and determined that the problem is unbounded.'
     nit: 457
   slack: array([ 5.05709050e+04,  1.62550964e+05, -2.51362949e+04,  6.88042749e+04,
        1.60126518e+04, -4.61710815e+07])
  status: 3
 success: False
       x: array([1.31420671e+03, 1.55091070e+03, 3.95832445e+03, 1.40957982e+03,
       5.49722044e+05, 4.48429731e+02, 8.35871538e+02, 1.38294908e+03,
       1.80175878e+03, 7.69135072e+02, 6.85498873e+02, 1.05877501e+03,
       7.71394213e+02, 2.51758204e+03, 4.97404821e+02, 1.15662639e+03,
       1.06586915e+03, 1.10316878e+03, 4.94291025e+03, 1.86883548e+02,
       2.27088751e+03, 9.63712722e+02, 7.71966714e+02, 8.47155878e+02,
       6.61724692e+02, 1.60805572e+03, 5.57657303e+05, 8.96334360e+02,
       4.52122318e+02, 2.10841300e+03, 1.06000454e+03, 6.17835819e+02,
       3.32849695e+03, 2.38615463e+03, 3.09398236e+03, 1.48792549e+02,
       1.28007034e

In [6]:
api.submit_order(symbol = 'AAPL',
                 qty = 3,
                 side = 'buy',
                 type = 'market',
                 time_in_force = 'day')

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': 'a192e8d2-b3b1-4a79-8054-dfd649e61a56',
    'created_at': '2020-04-03T21:32:04.784938Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '77439e17-4f58-401c-a38c-89029d04868c',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '3',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-03T21:32:04.777419Z',
    'symbol': 'AAPL',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-03T21:32:04.784938Z'})

In [ ]:
clock = api.get_clock()
if clock.is_open :
    #trade away! 

In [7]:
ff5

{0:             Mkt-RF   SMB   HML   RMW   CMA     RF
 Date                                             
 2015-03-09    0.37  0.09 -0.02  0.26 -0.03  0.000
 2015-03-10   -1.63  0.43 -0.47  0.09 -0.14  0.000
 2015-03-11   -0.04  0.62  0.52 -0.47 -0.03  0.000
 2015-03-12    1.28  0.40  0.50 -0.06  0.23  0.000
 2015-03-13   -0.57  0.19 -0.04 -0.10 -0.13  0.000
 ...            ...   ...   ...   ...   ...    ...
 2020-01-27   -1.56  0.27 -0.48 -0.59  0.05  0.006
 2020-01-28    1.02 -0.15 -0.42  0.17 -0.10  0.006
 2020-01-29   -0.10 -0.44 -0.95  0.25  0.19  0.006
 2020-01-30    0.34 -0.64  0.67 -0.39 -0.11  0.006
 2020-01-31   -1.74 -0.52 -0.34 -0.54 -0.69  0.006
 
 [1235 rows x 6 columns],
 'DESCR': 'F-F Research Data 5 Factors 2x3 daily\n-------------------------------------\n\nThis file was created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202001 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc.\n\n  0 : (1235 rows x 6 cols)'}